In [1]:
%load_ext watermark
%watermark -a 'Christian Schuhegger' -u -d -v -p numpy,scipy,pandas,matplotlib,seaborn,mxnet

Christian Schuhegger 
last updated: 2019-02-26 

CPython 3.6.8
IPython 7.3.0

numpy 1.14.6
scipy 1.2.0
pandas 0.24.1
matplotlib 3.0.2
seaborn 0.9.0
mxnet 1.3.1


In [2]:
%matplotlib inline
import numpy as np, scipy, scipy.stats as stats, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import sklearn, sklearn.pipeline, sklearn.model_selection, sklearn.preprocessing
import logging, time, datetime
import mxnet as mx
from mxnet import gluon, nd, autograd, metric
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(linewidth=1000)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

sns.set()

In [3]:
os.environ["CUDA_DEVICE_ORDER"]    = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ''

os.environ["KERAS_BACKEND"]        = "mxnet"
# image_data_format: channels_last

In [4]:
from keras import backend as K

Using MXNet backend


In [5]:
K.get_num_gpus()

1

In [6]:
mx.context.num_gpus()

0

In [7]:
K.mx.context.num_gpus()

0

In [8]:
import keras

In [9]:
keras.__version__

'2.2.4.1'

In [10]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

def display_graphs_side_by_side(*args):
    html_str='<table><tr>'
    for g in args:
        html_str += '<td>'
        html_str += g._repr_svg_()
        html_str += '</td>'
    html_str += '</tr></table>'
    display_html(html_str,raw=True)
    

display(HTML("<style>.container { width:70% !important; }</style>"))

In [11]:
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:%(name)s:%(levelname)s: %(message)s')
# log = logging.getLogger('std')

In [12]:
N_in       = 1000
N_subjects = 260 * 10 # 100000
N_subjects = 100000
W = stats.norm(loc=0, scale=1).rvs(size=(2,N_in), random_state=np.random.RandomState(42))
X = stats.norm(loc=0, scale=1).rvs(size=(N_subjects,N_in), random_state=np.random.RandomState(43))
y = np.sum(W[1:,:] * X + W[0,:], axis=1)

In [13]:
y.shape, X.shape

((100000,), (100000, 1000))

In [14]:
pd.Series(y).describe()

count    100000.000000
mean         19.234805
std          31.516783
min        -147.380827
25%          -2.102370
50%          19.273695
75%          40.475011
max         147.253509
dtype: float64

In [15]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.1, random_state = 42)

In [16]:
batch_size = 256
epochs     = 20

In [17]:
def create_tensorflow_keras_model(input_tensor):
    ACTIVATION = 'relu'

    net = input_tensor

    net = keras.layers.Dense(300, kernel_initializer=keras.initializers.glorot_normal(seed=42), activation=ACTIVATION, name='fc-1')(net)
    net = keras.layers.Dense(100, kernel_initializer=keras.initializers.glorot_normal(seed=42), activation=ACTIVATION, name='fc-2')(net)
    net = keras.layers.Dense(1, kernel_initializer=keras.initializers.glorot_normal(seed=42), activation=None, name='predictions')(net)

    return net

In [18]:
sym_X_in  = keras.layers.Input(shape=[X.shape[1]])
sym_y_out = create_tensorflow_keras_model(sym_X_in)

tf_keras_model = keras.models.Model(sym_X_in, sym_y_out, name='MLP')
tf_keras_model._context = [mx.cpu()]
# tf_keras_model = keras.utils.multi_gpu_model(tf_keras_model, gpus=0, cpu_merge=True, cpu_relocation=False)
tf_keras_model.compile("adam", "mean_squared_error", ["mean_absolute_error"])

time1 = time.time()
tf_keras_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)#, validation_split=0.1
time2 = time.time()

Epoch 1/20
 7424/90000 [=>............................] - ETA: 1s - loss: 964.1799 - mean_absolute_error: 24.5839 

/home/local/cs/local/install/anaconda3-5.3.1-Linux-x86_64/envs/mxnetkeras/lib/python3.6/site-packages/mxnet/module/bucketing_module.py:408: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.00390625). Is this intended?
  force_init=force_init)


90000/90000 [==============================] - 2s 19us/step - loss: 141.6004 - mean_absolute_error: 6.0683
Epoch 2/20
90000/90000 [==============================] - 2s 19us/step - loss: 3.7242 - mean_absolute_error: 1.5298
Epoch 3/20
90000/90000 [==============================] - 2s 19us/step - loss: 1.8966 - mean_absolute_error: 1.0925
Epoch 4/20
90000/90000 [==============================] - 2s 18us/step - loss: 1.1846 - mean_absolute_error: 0.8620
Epoch 5/20
90000/90000 [==============================] - 2s 18us/step - loss: 0.8239 - mean_absolute_error: 0.7187
Epoch 6/20
90000/90000 [==============================] - 2s 18us/step - loss: 0.6523 - mean_absolute_error: 0.6364
Epoch 7/20
90000/90000 [==============================] - 2s 19us/step - loss: 0.5820 - mean_absolute_error: 0.6010
Epoch 8/20
90000/90000 [==============================] - 2s 18us/step - loss: 0.6185 - mean_absolute_error: 0.6206
Epoch 9/20
90000/90000 [==============================] - 2s 19us/step - loss: 0.

In [19]:
print('time: {}'.format(time2-time1))

time: 32.88792848587036


In [20]:
y_tf_keras_pred = tf_keras_model.predict(X_test)
s = sklearn.metrics.mean_squared_error(y_test, y_tf_keras_pred)
s

1.5528236029114455

In [21]:
sklearn.metrics.explained_variance_score(y_test, y_tf_keras_pred)

0.998417783008194